#### SQL analytics and Dashbaord creation


In [0]:
%sql
select * from gold_category_performance limit 5

category_id,category_code,views,carts,purchases,revenue,cart_to_purchase_rate
2088750570935419494,construction.tools.painting,28,0,0,null,0.0
2106075725441269865,null,16077,291,181,7139.079999999997,62.19931271477663
2070005009256284935,null,16540,132,40,5659.320000000001,30.303030303030305
2053013565782098913,apparel.shoes,91873,2893,1896,254128.66999999998,65.53750432077429
2053013555413778833,null,15028,524,428,43564.22999999999,81.67938931297711


In [0]:
# create a moving average with event_date
%sql
WITH daily_sales AS (
    SELECT 
        event_date, 
        SUM(price) as daily_revenue
    FROM silver_events
    WHERE event_type = 'purchase'
    GROUP BY event_date
)
SELECT 
    event_date,
    daily_revenue,
    -- Calculate 7-Day Moving Average
    AVG(daily_revenue) OVER (
        ORDER BY event_date 
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) as revenue_7day_ma
FROM daily_sales
ORDER BY event_date ASC;

event_date,daily_revenue,revenue_7day_ma
2019-10-01,6275579.060000004,6275579.060000004
2019-10-02,6213628.530000011,6244603.795000007
2019-10-03,6233782.980000017,6240996.856666677
2019-10-04,8623058.190000022,6836512.1900000125
2019-10-05,7341094.46000001,6937428.6440000115
2019-10-06,6736895.7400000105,6904006.493333344
2019-10-07,6347448.089999998,6824498.150000009
2019-10-08,6819701.260000011,6902229.892857154
2019-10-09,6855193.41,6993882.018571438
2019-10-10,6665413.20999998,7055543.480000004


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
  category_code,
  round(SUM(revenue),2) AS total_revenue
FROM gold_category_performance
GROUP BY category_code
ORDER BY total_revenue DESC
LIMIT 10

category_code,total_revenue
electronics.smartphone,3.3485448998E8
null,5.28007829E7
electronics.video.tv,2.087912366E7
computers.notebook,1.965731313E7
electronics.clocks,1.136962062E7
appliances.kitchen.washer,1.046004616E7
electronics.audio.headphone,9208276.45
appliances.kitchen.refrigerators,8552451.19
appliances.environment.vacuum,4478737.03
electronics.tablet,3131062.01


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Conversion funnel by category
SELECT category_code,
  SUM(views) as views,
  SUM(purchases) as purchases,
  ROUND(SUM(purchases)*100.0/SUM(views), 2) as conversion_rate
FROM gold_category_performance
GROUP BY category_code
LIMIT 100;

category_code,views,purchases,conversion_rate
construction.tools.painting,8010,178,2.22
null,7077861,279354,3.95
apparel.shoes,511375,10055,1.97
apparel.shirt,35638,127,0.36
furniture.living_room.chair,28916,947,3.28
construction.tools.saw,60353,2519,4.17
electronics.tablet,139718,7686,5.50
sport.bicycle,53956,931,1.73
construction.tools.light,17756,148,0.83
construction.tools.screw,59,1,1.69


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Customer tiers/segmentation
SELECT
  CASE WHEN cnt >= 10 THEN 'VIP'
       WHEN cnt >= 5 THEN 'Loyal'
       ELSE 'Regular' END as tier,
  COUNT(*) as customers,
  AVG(total_spent) as avg_ltv
FROM (SELECT user_id, COUNT(*) cnt, SUM(price) total_spent
      FROM silver_events WHERE event_type='purchase' GROUP BY user_id)
GROUP BY tier;

tier,customers,avg_ltv
Regular,631022,429.1299216192347
VIP,19869,7070.269982887916
Loyal,46578,2014.9949562024972


Databricks visualization. Run in Databricks to view.